In [2]:
def take_genes_from_bed(name_of_bed: str):
    names = ['chr', 'start', 'stop', 'gene', 'to_drop', 'strand']
    df = pd.read_csv(name_of_bed, sep="\t",
                     header=None,
                     names=names,
                     skiprows=1)

    df = df.drop('to_drop', axis=1)
    return np.unique(df.gene.values)

In [3]:
import pandas as pd
import numpy as np

In [4]:
names = ['chr', 'position', 'coverage']
names2 = ['chr', 'start', 'stop', 'gene', 'to_drop', 'strand']
name_of_bed = 'DHS-003Z.primers-150bp.bed'

df = pd.read_csv('samtools_depth', sep="\t", names=names)
df2 = pd.read_csv(name_of_bed, sep="\t",
                     header=None,
                     names=names2,
                     skiprows=1)

df2 = df2.drop('to_drop', axis=1)

In [30]:
np.diff(take_genes_from_bed(name_of_bed), np.unique(df.gene.values))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [10]:
df.head()

,chr,position,coverage,gene
0,chr1,12918806,0,PRAMEF2
1,chr1,12918807,0,PRAMEF2
2,chr1,12918808,0,PRAMEF2
3,chr1,12918809,0,PRAMEF2
4,chr1,12918810,0,PRAMEF2


In [6]:
genes_bound = dict()
for gene in take_genes_from_bed(name_of_bed):
    df3 = df2[df2['gene'] == gene]
    genes_bound[gene] = (df3.iloc[0]['start'],
                         df3.iloc[len(df3)-1]['stop'])
print(genes_bound)

    

{'ABL1': (133589700, 133760909), 'ADA': (43248392, 43280253), 'ANKRD26': (27283962, 27389286), 'ASXL1': (30946469, 31025251), 'ASXL2': (25964891, 26101127), 'ATM': (108098348, 108236239), 'ATRX': (76763984, 77041554), 'BCL6': (187440239, 187451505), 'BCOR': (39911348, 39937202), 'BCORL1': (129139153, 129190198), 'BCR': (23523099, 23657725), 'BIRC3': (102195341, 102207746), 'BLM': (91290617, 91358514), 'BRAF': (140415740, 140624529), 'BRCA1': (41197629, 41276132), 'BRCA2': (32890548, 32972912), 'C17orf97': (260055, 263968), 'CALR': (13049435, 13054590), 'CARD11': (2946335, 2998223), 'CBL': (119077207, 119170510), 'CBLB': (105377808, 105588286), 'CBLC': (45281141, 45303733), 'CDKN2A': (21968219, 21994412), 'CEBPA': (33792227, 33793324), 'CHEK2': (29083889, 29130731), 'CREBBP': (3777689, 3929917), 'CRLF2': (1314863, 1331555), 'CSF1R': (149433540, 149466038), 'CSF3R': (36931670, 36945107), 'CTCF': (67644711, 67671811), 'CUX1': (101459304, 101926433), 'DAXX': (33286488, 33290742), 'DDX41': 

In [37]:
def add_gene_name(df_to_add, df_from_add): # TODO OPTIMIZE
    genes = np.array(['0' * 10 for i in range(len(df_to_add))])
    for pos in range(len(df_to_add)):
        for index, row in df_from_add.iterrows():
            if (row['start'] + 1 <= df_to_add.position[pos] <= row['stop']) and\
                    (df_to_add.chr[pos] == row['chr']):
                genes[pos] = row['gene']
                break
    return genes 

In [7]:
                                                                
def add_gene_name_optimized(df_to_add, df_from_add):            
    percents = 0                                                
    percents_5 = int(len(df_to_add) * 0.05)                     
    genes = np.array(['0' * 10 for i in range(len(df_to_add))]) 
    for index, row in df_to_add.iterrows():                     
        if index % percents_5 == 0:                             
            print('{} % have done..'.format(percents))          
            percents += 5                                       
        for gene, bounds in genes_bound.items():                
            if bounds[0] <= row['position'] + 1 <= bounds[1]+ 1):    
                genes[index] = gene
                break
            
    print("number of genes: {}".format(len(np.unique(genes))))  
    return genes

In [8]:
print('Hello')
genes = add_gene_name_optimized(df, df2)
df['gene'] = genes
print('Good Bye')

Hello
0 % have done..
5 % have done..
10 % have done..
15 % have done..
20 % have done..
25 % have done..
30 % have done..
35 % have done..
40 % have done..
45 % have done..
50 % have done..
55 % have done..
60 % have done..
65 % have done..
70 % have done..
75 % have done..
80 % have done..
85 % have done..
90 % have done..
95 % have done..
100 % have done..
number of genes: 138
Good Bye


In [9]:
df[df['gene'] == '0000000000']

,chr,position,coverage,gene
15013,chr1,115251146,0,0000000000
15014,chr1,115251147,0,0000000000
15015,chr1,115251148,0,0000000000
15016,chr1,115251149,0,0000000000
15017,chr1,115251150,0,0000000000
...,...,...,...,...
520823,chrX,123480521,0,0000000000
520824,chrX,123480522,0,0000000000
520825,chrX,123480523,0,0000000000
520826,chrX,123480524,0,0000000000


In [29]:
df2[df2['start'] == '12921696']


,chr,start,stop,gene,strand
